In [3]:
import os
import cv2
import numpy as np
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks, rotate
from skimage.color import rgb2gray
from matplotlib import pyplot as plt
import itertools
import tensorflow as tf
from tkinter import Tk, filedialog, Button, Label
from PIL import Image, ImageTk

# Функция для обработки и отображения изображения
def process_image(image_path):
    # Чтение изображения
    image = cv2.imread(image_path)

    # Преобразование изображения в оттенки серого и нахождение краев
    grayscale = rgb2gray(image)
    edges = canny(grayscale, sigma=3.0)

    # Применение преобразования Хафа
    out, angles, distances = hough_line(edges)
    h, theta, d = out, angles, distances

    # Расчет угла поворота
    angle_step = 0.5 * np.diff(theta).mean()
    d_step = 0.5 * np.diff(d).mean()
    bounds = [np.rad2deg(theta[0] - angle_step),
              np.rad2deg(theta[-1] + angle_step),
              d[-1] + d_step, d[0] - d_step]
    _, angles_peaks, _ = hough_line_peaks(out, angles, distances, num_peaks=20)
    angle = np.mean(np.rad2deg(angles_peaks))
    if 0 <= angle <= 90:
        rot_angle = angle - 90
    elif -45 <= angle < 0:
        rot_angle = angle - 90
    elif -90 <= angle < -45:
        rot_angle = 90 + angle
    if abs(rot_angle) > 20:
        rot_angle = 0

    # Поворот изображения
    rotated = rotate(image, rot_angle, resize=True) * 255
    rotated = rotated.astype(np.uint8)

    # Обрезка изображения
    rotated1 = rotated[:, :, :]
    if rotated.shape[1] / rotated.shape[0] < 2:
        minus = np.abs(int(np.sin(np.radians(rot_angle)) * rotated.shape[0]))
        rotated1 = rotated[minus:-minus, :, :]

    # Преобразование изображения для улучшения контрастности
    lab = cv2.cvtColor(rotated1, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    return image, final

# Функция для выбора файла
def select_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        original_image, final_image = process_image(file_path)
        show_images(original_image, final_image)

# Функция для отображения изображений
def show_images(original_image, final_image):
    # Отображение исходного изображения
    image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(image_rgb)
    img_tk = ImageTk.PhotoImage(image=img_pil)
    original_label.config(image=img_tk)
    original_label.image = img_tk

    # Отображение обработанного изображения
    image_rgb = cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(image_rgb)
    img_tk = ImageTk.PhotoImage(image=img_pil)
    processed_label.config(image=img_tk)
    processed_label.image = img_tk

# Создание основного окна
root = Tk()
root.title("Image Processor")

# Кнопка для выбора файла
btn = Button(root, text="Select Image", command=select_file)
btn.pack()

# Метка для отображения исходного изображения
original_label = Label(root)
original_label.pack(side="left", padx=10, pady=10)

# Метка для отображения обработанного изображения
processed_label = Label(root)
processed_label.pack(side="right", padx=10, pady=10)

# Запуск основного цикла
root.mainloop()
